# sklearn一些小技巧的记录（pipeline...）

## `LabelEncoder`
简单来说 LabelEncoder 是对不连续的数字或者文本进行编号：

In [1]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit([1,5,67,100])
le.transform([1,1,100,67,5])

array([0, 0, 3, 2, 1], dtype=int64)

# `OneHotEncoder`
OneHotEncoder 用于将表示分类的数据扩维：

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit([[1], [2], [3], [4]])
ohe.transform([[2], [3], [1], [4]]).toarray()

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

正如 keras 中的 `keras.utils.to_categorical(y_train, num_classes)`

# `sklearn.model_selection.train_test_split` 随机划分训练集和测试集
一般形式： 
`train_test_split`是交叉验证中常用的函数，功能是从样本中随机的按比例选取`train data`和`test data`，形式为：

```py
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    train_data, train_target, test_size=0.4, random_state=0)
```

参数解释： 
- `train_data`：所要划分的样本特征集 
- `train_target`：所要划分的样本结果 
- `test_size`：样本占比，如果是整数的话就是样本的数量 
- `random_state`：是随机数的种子。 
- 随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填`1`，其他参数一样的情况下你得到的随机数组是一样的。但填`0`或不填，每次都会不一样。

随机数的产生取决于种子，随机数和种子之间的关系遵从以下两个规则： 
- 种子不同，产生不同的随机数；种子相同，即使实例不同也产生相同的随机数。

```py
from sklearn.cross_validation import train_test_split
train = loan_data.iloc[0: 55596, :]
test = loan_data.iloc[55596:, :]
# 避免过拟合，采用交叉验证，验证集占训练集20%，固定随机种子（random_state)
train_X, test_X, train_y, test_y = train_test_split(
    train, target, test_size=0.2, random_state=0)
train_y = train_y['label']
test_y = test_y['label']
```

# `pipeline`
`pipeline` 实现了对全部步骤的流式化封装和管理，可以很方便地使参数集在新数据集上被重复使用。

`pipeline` 可以用于下面几处：
- 模块化 `Feature Transform`，只需写很少的代码就能将新的 `Feature` 更新到训练集中。
- 自动化 `Grid Search`，只要预先设定好使用的 `Model` 和参数的候选，就能自动搜索并记录最佳的 `Model`。
- 自动化 `Ensemble Generation`，每隔一段时间将现有最好的 `K `个 `Model` 拿来做 `Ensemble`。

管道机制在机器学习算法中得以应用的根源在于，参数集在新数据集（比如测试集）上的重复使用。

管道机制实现了对全部步骤的流式化封装和管理（streaming workflows with pipelines）。

注意：管道机制更像是编程技巧的创新，而非算法的创新。

接下来我们以一个具体的例子来演示sklearn库中强大的Pipeline用法：
问题是要对数据集 Breast Cancer Wisconsin 进行分类， 
它包含 $569$ 个样本，第一列 `ID`，第二列类别(`M=恶性肿瘤，B=良性肿瘤`)， 
第 `3-32` 列是实数值的特征。

In [9]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
# Breast Cancer Wisconsin dataset
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/'
    'breast-cancer-wisconsin/wdbc.data', header=None)


X, y = df.values[:, 2:], df.values[:, 1]

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
encoder.transform(['M', 'B'])

array([1, 0], dtype=int64)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

我们要用 Pipeline 对训练集和测试集进行如下操作：
- 先用 StandardScaler 对数据集每一列做标准化处理，（是 transformer）
- 再用 PCA 将原始的 `30` 维度特征压缩的 `2` 维度，（是 transformer）
- 最后再用模型 LogisticRegression。（是 Estimator）
- 调用 Pipeline 时，输入由元组构成的列表，每个元组第一个值为变量名，元组第二个元素是 sklearn 中的 transformer 或 Estimator。

注意中间每一步是 transformer，即它们必须包含 `fit` 和 `transform` 方法，或者 `fit_transform`。 
最后一步是一个 Estimator，即最后一步模型要有 `fit` 方法，可以没有 `transform`方法。
然后用 Pipeline.fit对训练集进行训练，`pipe_lr.fit(X_train, y_train)` 
再直接用 Pipeline.score 对测试集进行预测并评分 `pipe_lr.score(X_test, y_test)`：

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

pipe_lr = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression(random_state=1))
                    ])
pipe_lr.fit(X_train, y_train)
print('Test accuracy: %.3f' % pipe_lr.score(X_test, y_test))

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Test accuracy: 0.921


Pipeline对象接受二元tuple构成的list，每一个二元 tuple 中的第一个元素为 arbitrary identifier string，我们用以获取（access）Pipeline object 中的 individual elements，二元 tuple 中的第二个元素是 scikit-learn与之相适配的transformer 或者 estimator。